# Testing a hypothesis about the effect of recession on housing prices in university towns as compared to other towns
## Tanishk Sachdeva
To test the hypotheses that the university towns have their mean housing prices less effected by recessions, the notebook runs a t-test using ```scipy.stats.ttest_ind``` to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom vs the same price ratio for other towns. 
 
This is part of an assignment for the the online course [Introduction to Data Science in Python](https://www.coursera.org/learn/python-data-analysis). The data files used here can be found at:
* The list of
[university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States)
from wikipedia has been copy and pasted into the file ```university_towns.txt``` in the repo. 
* The housing data from the 
[Zillow research data site](http://www.zillow.com/research/data/) stored in the file ```City_Zhvi_AllHomes.csv```.
* The [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States from the Bureau of Economic Analysis, US Department of Commerce, stored in the file ```gdplev.xls```

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

In [2]:
def get_list_of_university_towns():
    '''Returns a DataFrame with multi index consisting of the towns and the states from the 
    university_towns.txt list.'''
    text_file = open("university_towns.txt")
    # Make a dictionary with key as the line number of the states that have "[edit]" after them 
    # and value as state name itself without "[edit]" 
    State = {idx: lines.strip().replace("[edit]", "") 
             for idx,lines in enumerate(text_file) if "edit" in lines} 
    State = pd.Series(State) # Convert the dictionary to a series
    university_towns = pd.read_csv("university_towns.txt", sep = "\n", header = None, 
                                   names = ["RegionName"])
    university_towns["State"] = State # Add the above series as a column in dataframe
    university_towns = university_towns.fillna(method = 'ffill') # Forward fill for the "State" Column
    # Drop all rows that has state names in the column "RegionName"
    university_towns = university_towns.drop(State.index) 
    university_towns["RegionName"] = list(map(lambda x: x.split("(")[0].rstrip(), 
                                              university_towns["RegionName"])) 
    university_towns = university_towns.set_index(["State", "RegionName"])
    return university_towns
print("List of university towns: \n")
get_list_of_university_towns()

List of university towns: 



Empty DataFrame
Columns: []
Index: [(Alabama, Auburn), (Alabama, Florence), (Alabama, Jacksonville), (Alabama, Livingston), (Alabama, Montevallo), (Alabama, Troy), (Alabama, Tuscaloosa), (Alabama, Tuskegee), (Alaska, Fairbanks), (Arizona, Flagstaff), (Arizona, Tempe), (Arizona, Tucson), (Arkansas, Arkadelphia), (Arkansas, Conway), (Arkansas, Fayetteville), (Arkansas, Jonesboro), (Arkansas, Magnolia), (Arkansas, Monticello), (Arkansas, Russellville), (Arkansas, Searcy), (California, Angwin), (California, Arcata), (California, Berkeley), (California, Chico), (California, Claremont), (California, Cotati), (California, Davis), (California, Irvine), (California, Isla Vista), (California, University Park, Los Angeles), (California, Merced), (California, Orange), (California, Palo Alto), (California, Pomona), (California, Redlands), (California, Riverside), (California, Sacramento), (California, University District, San Bernardino), (California, San Diego), (California, San Luis Obispo), (California, Santa Barbara), (California, Santa Cruz), (California, Turlock), (California, Westwood, Los Angeles), (California, Whittier), (Colorado, Alamosa), (Colorado, Boulder), (Colorado, Durango), (Colorado, Fort Collins), (Colorado, Golden), (Colorado, Grand Junction), (Colorado, Greeley), (Colorado, Gunnison), (Colorado, Pueblo, Colorado), (Connecticut, Fairfield), (Connecticut, Middletown), (Connecticut, New Britain), (Connecticut, New Haven), (Connecticut, New London), (Connecticut, Storrs), (Connecticut, Willimantic), (Delaware, Dover), (Delaware, Newark), (Florida, Ave Maria), (Florida, Boca Raton), (Florida, Coral Gables), (Florida, DeLand), (Florida, Estero), (Florida, Gainesville), (Florida, Orlando), (Florida, Sarasota), (Florida, St. Augustine), (Florida, St. Leo), (Florida, Tallahassee), (Florida, Tampa), (Georgia, Albany), (Georgia, Athens), (Georgia, Atlanta), (Georgia, Carrollton), (Georgia, Demorest), (Georgia, Fort Valley), (Georgia, Kennesaw), (Georgia, Milledgeville), (Georgia, Mount Vernon), (Georgia, Oxford), (Georgia, Rome), (Georgia, Savannah), (Georgia, Statesboro), (Georgia, Valdosta), (Georgia, Waleska), (Georgia, Young Harris), (Hawaii, Manoa), (Idaho, Moscow), (Idaho, Pocatello), (Idaho, Rexburg), (Illinois, Carbondale), (Illinois, Champaign–Urbana), (Illinois, Charleston), (Illinois, DeKalb), (Illinois, Edwardsville), ...]

[517 rows x 0 columns]

A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.

In [3]:
def get_quarter_before_recession():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 5)
    GDP.drop(GDP.index[:214], inplace = True)
    GDP.dropna(axis = 1, how = 'all', inplace = True)
    GDP = GDP[[0,2]]
    GDP.columns = ["Quarters", 'GDP in billions in current dollars']
    GDP.set_index("Quarters", inplace = True)
    GDP_diff = GDP.diff() 
    for i, quarter in enumerate(GDP_diff.index):
        if (GDP_diff.iloc[i+1] < 0).bool() and (GDP_diff.iloc[i+2] < 0).bool():
            return quarter
    return None  
print("Quarter before recession: ", get_quarter_before_recession())

Quarter before recession:  2008q2


In [4]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 5)
    GDP.drop(GDP.index[:214], inplace = True)
    GDP.dropna(axis = 1, how = 'all', inplace = True)
    GDP = GDP[[0,2]]
    GDP.columns = ["Quarters", 'GDP in billions in current dollars']
    GDP.set_index("Quarters", inplace = True)
    GDP_diff = GDP.diff()
    flag = 0
    for i, quarter in enumerate(GDP_diff.index):
        if (flag == 1) and (GDP_diff.iloc[i-1] > 0).bool() and (GDP_diff.iloc[i] > 0).bool():
            return quarter
        if (GDP_diff.iloc[i] < 0).bool() and (GDP_diff.iloc[i+1] < 0).bool():
            flag = 1
    return None
print("Recession end: ", get_recession_end())

Recession end:  2009q4


A _recession bottom_ is the quarter within a recession which had the lowest GDP.

In [5]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 5)
    GDP.drop(GDP.index[:214], inplace = True)
    GDP.dropna(axis = 1, how = 'all', inplace = True)
    GDP = GDP[[0,2]]
    GDP.columns = ["Quarters", 'GDP in billions in current dollars']
    GDP.set_index("Quarters", inplace = True)
    GDP_diff = GDP.diff()
    flag = 0
    for i, quarter in enumerate(GDP_diff.index):
        if (flag == 1) and (GDP_diff.iloc[i] > 0).bool() and (GDP_diff.iloc[i+1] > 0).bool():
            end_idx = i+1
            break
        if (flag == 0) and (GDP_diff.iloc[i] < 0).bool() and (GDP_diff.iloc[i+1] < 0).bool():
            flag = 1
            start_idx = i
    GDP_recession = GDP.iloc[start_idx: end_idx + 1]
    idx = GDP_recession.idxmin()
    return idx[0]
print("Recession bottom: ", get_recession_bottom())

Recession bottom:  2009q2


A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.

In [6]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    all_homes = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
              'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 
              'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 
              'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 
              'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 
              'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 
              'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 
              'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 
              'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 
              'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 
              'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 
              'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 
              'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 
              'VA': 'Virginia'}
    # Replaces the abbreviations with the names of the states
    all_homes["State"].replace(states, inplace = True) 
    all_homes = all_homes.set_index(["State","RegionName"])
    all_homes = all_homes.iloc[:, 49:250] # Discards irrelavant columns
    
    def quarters(col):
        if col.endswith(("01", "02", "03")):
            s = col[:4] + "q1"
        elif col.endswith(("04", "05", "06")):
            s = col[:4] + "q2"
        elif col.endswith(("07", "08", "09")):
            s = col[:4] + "q3"
        else:
            s = col[:4] + "q4"
        return s  
    # Groups the monthly columns into quarters using mean value of the four monthly columns
    housing = all_homes.groupby(quarters, axis = 1).mean() 
    housing = housing.sort_index()
    return housing
housing = convert_housing_data_to_quarters()
print("Columns: \n", housing.columns)
print("# Rows: ", len(housing))
housing.head()

Columns: 
 Index(['2000q1', '2000q2', '2000q3', '2000q4', '2001q1', '2001q2', '2001q3',
       '2001q4', '2002q1', '2002q2', '2002q3', '2002q4', '2003q1', '2003q2',
       '2003q3', '2003q4', '2004q1', '2004q2', '2004q3', '2004q4', '2005q1',
       '2005q2', '2005q3', '2005q4', '2006q1', '2006q2', '2006q3', '2006q4',
       '2007q1', '2007q2', '2007q3', '2007q4', '2008q1', '2008q2', '2008q3',
       '2008q4', '2009q1', '2009q2', '2009q3', '2009q4', '2010q1', '2010q2',
       '2010q3', '2010q4', '2011q1', '2011q2', '2011q3', '2011q4', '2012q1',
       '2012q2', '2012q3', '2012q4', '2013q1', '2013q2', '2013q3', '2013q4',
       '2014q1', '2014q2', '2014q3', '2014q4', '2015q1', '2015q2', '2015q3',
       '2015q4', '2016q1', '2016q2', '2016q3'],
      dtype='object')
# Rows:  10730


2000q1         2000q2         2000q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   

                            2000q4         2001q1         2001q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   

                            2001q3         2001q4         2002q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   

                            2002q2    ...            2014q2         2014q3  \
State   RegionName                    ...                                    
Alabama Adamsville    73133.333333    ...      77066.666667   75966.666667   
        Alabaster    128000.000000    ...     147133.333333  147633.333333   
        Albertville   76366.666667    ...      84033.333333   84766.666667   
        Arab          87700.000000    ...     113366.666667  111700.000000   
        Ardmore                NaN    ...     140533.333333  139566.666667   

                       2014q4         2015q1         2015q2         2015q3  \
State   RegionName                                                           
Alabama Adamsville    71900.0   71666.666667   73033.333333   73933.333333   
        Alabaster    148700.0  148900.000000  149566.666667  150366.666667   
        Albertville   86800.0   88466.666667   89500.000000   90233.333333   
        Arab         111600.0  110166.666667  109433.333333  110900.000000   
        Ardmore      140900.0  143233.333333  143000.000000  144600.000000   

                            2015q4         2016q1         2016q2    2016q3  
State   RegionName                                                          
Alabama Adamsville    73866.666667   74166.666667   74933.333333   74700.0  
        Alabaster    151733.333333  153466.666667  155100.000000  155850.0  
        Albertville   91366.666667   92000.000000   92466.666667   92200.0  
        Arab         112233.333333  110033.333333  110100.000000  112000.0  
        Ardmore      143966.666667  142566.666667  143233.333333  141950.0  

[5 rows x 67 columns]

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Running a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)


In [7]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Returns the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p is the 
    exact p value returned from scipy.stats.ttest_ind(). The value for better is 
    either "university town" or "non-university town" depending on which has a 
    lower mean price ratio (which is equivilent to a reduced market loss).'''
    housing = convert_housing_data_to_quarters()
    university_towns = get_list_of_university_towns()
    quarter_before_recession = get_quarter_before_recession()    
    recession_bottom = get_recession_bottom()
    housing = housing[[quarter_before_recession, recession_bottom]]
    housing["price_ratio"] = housing[quarter_before_recession].div(housing[recession_bottom])
    housing = housing.dropna()
    university_housing = pd.merge(university_towns, housing, how = "inner", 
                                  left_index = True, right_index = True)
    non_university_housing = housing[~housing.index.isin(university_housing.index)]
    from scipy import stats
    t_stat, p_value = stats.ttest_ind(university_housing["price_ratio"], 
                                      non_university_housing["price_ratio"])
    different = True
    if p_value < 0.01:
        different = True
    else:
        different = False
    if t_stat < 0:
        better = "university town"
    else:
        better = "non-university town"
    return (different, p_value, better)
run_ttest()

(True, 0.002724063704761164, 'university town')